In [ ]:

# This script loads financial and operational KPI data from Excel files,
# merges them by date, cleans and prepares the dataset,
# and performs comprehensive trend analysis on the revenue KPI.
# The trend analysis includes calculating Compound Annual Growth Rate (CAGR),
# Year-over-Year (YoY) growth, linear regression trend slope and R²,
# and decomposing the revenue time series into trend, seasonal, and residual components.
# Finally, it visualizes the revenue and its decomposition components over time.

import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

# Load financial KPI Excel file
financial_df = pd.read_excel("data/financial_kpis.xlsx")

# Load operational KPI Excel file
operational_df = pd.read_excel("data/operational_kpis.xlsx")

# Standardize column names
financial_df.columns = financial_df.columns.str.lower().str.strip()
operational_df.columns = operational_df.columns.str.lower().str.strip()

# Convert 'date' columns to datetime
financial_df["date"] = pd.to_datetime(financial_df["date"])
operational_df["date"] = pd.to_datetime(operational_df["date"])

# Merge financial and operational KPI data on 'date'
df = pd.merge(financial_df, operational_df, on="date", how="inner")

# Remove duplicate rows
df = df.drop_duplicates()

# Sort by date for correct time series order
df = df.sort_values("date")

# Fill missing numeric values forward then backward
numeric_cols = df.select_dtypes(include="number").columns.tolist()
for col in numeric_cols:
    df[col] = df[col].ffill().bfill()

# Set 'date' as index for time series operations
df = df.set_index("date")


# Trend analysis: CAGR calculation function

def calculate_cagr(series):
    n_years = (series.index[-1] - series.index[0]).days / 365.25
    return (series.iloc[-1] / series.iloc[0]) ** (1 / n_years) - 1

# Trend analysis: Calculate Year-over-Year (YoY) growth for revenue

df["yoy_growth"] = df["revenue"].pct_change(periods=12)

# Trend analysis: Linear regression to find trend slope and R²

def linear_regression_trend(group):
    X = group.index.map(pd.Timestamp.toordinal).values.reshape(-1, 1)
    y = group["revenue"].values
    model = LinearRegression().fit(X, y)
    return pd.Series({"slope": model.coef_[0], "r_squared": model.score(X, y)})

trend_stats = linear_regression_trend(df)
print("Linear regression trend stats:")
print(trend_stats)


# Time series decomposition of revenue

df = df.asfreq("M")  # ensure monthly frequency for decomposition
decomp = seasonal_decompose(df["revenue"], model="additive", period=12)
df["trend"] = decomp.trend
df["seasonal"] = decomp.seasonal
df["residual"] = decomp.resid

# Plot original revenue and decomposed components

fig, axs = plt.subplots(4, 1, figsize=(12, 10), sharex=True)
axs[0].plot(df.index, df["revenue"])
axs[0].set_title("Revenue Over Time")
axs[1].plot(df.index, df["trend"])
axs[1].set_title("Trend Component")
axs[2].plot(df.index, df["seasonal"])
axs[2].set_title("Seasonal Component")
axs[3].plot(df.index, df["residual"])
axs[3].set_title("Residual Component")
plt.tight_layout()
plt.show()

# Trend analysis: Calculate and print CAGR

cagr = calculate_cagr(df["revenue"])
print(f"CAGR: {cagr:.2%}")
